# 2D Integrator

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid', palette='hls')

from FPE.integrator_2D import FPE_integrator_2D


In [3]:
D = 1.0
dx = 0.1
dy = 0.1
dt = 0.01
xArray = np.arange(-1, 1, dx)
yArray = np.arange(-1, 1, dy)

fpe2 = FPE_integrator_2D(D, dt, dx, dy, xArray, yArray, boundaryCond='periodic', output=True)




Initializing diffusion term integration matrix...

		Using Crank-Nicolson integration scheme...
		Initializing integration matrices for diffusion

